# Projeto deep learning

# Fine tuning da EfficientNet usando o dataset COVIDGR

## Step 1: Setup Google Drive, Libraries and GPU

In [ ]:
# Check GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 10 12:18:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
torch.cuda.is_available()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 2: Data Loading and Augmentation
Define Dataset Paths: Set up the paths to the P and N folders for loading images.

Data Augmentation: Implement the augmentation protocol, including random resized cropping, color jittering, color dropping, and Gaussian blurring.

Load Dataset: Use torchvision.datasets.ImageFolder with the custom transformations.

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
import torch

# Path to the dataset
dataset_path = '/content/drive/MyDrive/Pós-Grad/2024.2/Deep Learning/Projeto/Datasets/COVIDGR_1.0'
positive_path = os.path.join(dataset_path, 'P')
negative_path = os.path.join(dataset_path, 'N')

# Data transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.3, 0.9), ratio=(3/4, 4/3)),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
        transforms.RandomGrayscale(p=0.2)
    ]),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=25, sigma=(0.1, 2.0))], p=0.5),
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# Load dataset with ImageFolder
dataset = datasets.ImageFolder(root=dataset_path, transform=train_transform)

In [ ]:
dataset

Dataset ImageFolder
    Number of datapoints: 852
    Root location: /content/drive/MyDrive/Pós-Grad/2024.2/Deep Learning/Projeto/Datasets/COVIDGR_1.0
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.3, 0.9), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               RandomApply(
               p=0.5
               ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5), saturation=(0.5, 1.5), hue=(-0.1, 0.1))
               RandomGrayscale(p=0.2)
           )
               RandomApply(
               p=0.5
               GaussianBlur(kernel_size=(25, 25), sigma=(0.1, 2.0))
           )
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
           )

In [ ]:
# Step 4: Map labels: 'P' to 1, 'N' to 0
dataset.class_to_idx = {'N': 0, 'P': 1}
dataset

Dataset ImageFolder
    Number of datapoints: 852
    Root location: /content/drive/MyDrive/Pós-Grad/2024.2/Deep Learning/Projeto/Datasets/COVIDGR_1.0
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.3, 0.9), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
               RandomApply(
               p=0.5
               ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5), saturation=(0.5, 1.5), hue=(-0.1, 0.1))
               RandomGrayscale(p=0.2)
           )
               RandomApply(
               p=0.5
               GaussianBlur(kernel_size=(25, 25), sigma=(0.1, 2.0))
           )
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
           )

# Step 3: Data Splitting
90-10 Train-Test Split and Train-Validation Split.

In [ ]:
# Splitting the dataset
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
batch_size = 256
print(train_size, batch_size)

# Split off the test set
train_val_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader for the test set (held out)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

766 256


In [ ]:
len(dataset) * 0.9

766.8000000000001

# Step 4: Model Setup
Load Pretrained EfficientNet: Load EfficientNet with ImageNet pre-trained weights and adapt the final layer for binary classification.

Define Optimizer: Set up the SGD optimizer with momentum.

Hyperparameter Grid Search: You’ll need to run a grid search loop over learning rates and weight decay values.

In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn
import torch.optim as optim

def get_model():
    model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)  # Load pre-trained EfficientNet B0
    for param in model.parameters():
        param.requires_grad = False  # Freeze all layers
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # Modify final layer adding a linear binary classifier
    return model

## Step 6: Training and Validation
Define Training and Evaluation Loops: Track metrics like precision, recall, accuracy, and F1-score per epoch.

Cross-Validation: Implement 5-fold cross-validation, recording average and standard deviation metrics.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time
from sklearn.model_selection import KFold


def calculate_metrics(true_labels, predictions):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    return accuracy, precision, recall, f1


def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    train_losses, val_losses = [], []
    best_val_f1 = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()  # Adjust learning rate

        # Validation phase
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Metrics
        accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
        train_losses.append(running_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))

        if f1 > best_val_f1:
            best_val_f1 = f1

        print(f"[{device}] Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss / len(train_loader):.4f}, "
              f"Val Loss: {val_loss / len(val_loader):.4f}, F1 Score: {f1:.4f}")

    return best_val_f1

# Evaluate model on the test set
def evaluate_model(model, test_loader):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')
    return accuracy, precision, recall, f1


def grid_search(train_loader, val_loader, learning_rates, weight_decays, num_epochs):
    best_model = None
    best_f1 = 0
    best_params = {}
    for lr in learning_rates:
        for wd in weight_decays:
            model = get_model()
            optimizer = optim.SGD(model.classifier[1].parameters(), lr=lr, weight_decay=wd, momentum=0.9)
            scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: min(1.0, (epoch + 1) / 10))
            criterion = nn.CrossEntropyLoss()
            print(f"\nTraining with lr={lr}, weight_decay={wd}")
            f1_score = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)
            if f1_score > best_f1:
                best_f1 = f1_score
                best_model = model
                best_params = {"learning_rate": lr, "weight_decay": wd}
    print(f"\nBest Model F1: {best_f1} with params {best_params}")
    return best_model, best_params

# Step 9: 5-Fold Cross-Validation
def cross_validation(best_params, dataset, test_loader, num_epochs=50, folds=5):
    fold_metrics = []
    kfold = KFold(n_splits=folds, shuffle=True, random_state=100)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"\nStarting fold {fold + 1}/{folds}")

        # Split dataset indices for training and validation
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Create DataLoaders for this fold
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        model = get_model()
        optimizer = optim.SGD(model.classifier[1].parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'], momentum=0.9)
        scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: min(1.0, (epoch + 1) / 10))
        criterion = nn.CrossEntropyLoss()

        start_time = time.time()
        _ = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)
        end_time = time.time()

        # Evaluate on test set
        accuracy, precision, recall, f1 = evaluate_model(model, test_loader)
        fold_metrics.append((accuracy, precision, recall, f1, end_time - start_time))

    return np.array(fold_metrics)

Run grid search to get best params


In [ ]:
# Dataloaders
val_size = int(0.1 * len(train_val_dataset))
train_size = len(train_val_dataset) - val_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Hyperparameters
learning_rates = [1e-2, 1e-3, 1e-4]
weight_decays = [1e-3, 1e-4, 1e-5]
num_epochs = 50

In [ ]:
# Grid search
best_model, best_params = grid_search(train_loader, val_loader, learning_rates, weight_decays, num_epochs)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 187MB/s]



Training with lr=0.01, weight_decay=0.001
[cuda] Epoch 1/50, Train Loss: 0.6991, Val Loss: 0.6915, F1 Score: 0.5301
[cuda] Epoch 2/50, Train Loss: 0.6971, Val Loss: 0.7263, F1 Score: 0.6422
[cuda] Epoch 3/50, Train Loss: 0.6945, Val Loss: 0.7373, F1 Score: 0.6607
[cuda] Epoch 4/50, Train Loss: 0.6816, Val Loss: 0.7128, F1 Score: 0.6667
[cuda] Epoch 5/50, Train Loss: 0.6713, Val Loss: 0.6953, F1 Score: 0.6105
[cuda] Epoch 6/50, Train Loss: 0.6519, Val Loss: 0.7020, F1 Score: 0.4110
[cuda] Epoch 7/50, Train Loss: 0.6452, Val Loss: 0.6959, F1 Score: 0.4444
[cuda] Epoch 8/50, Train Loss: 0.6152, Val Loss: 0.6784, F1 Score: 0.4308
[cuda] Epoch 9/50, Train Loss: 0.6101, Val Loss: 0.6781, F1 Score: 0.5143
[cuda] Epoch 10/50, Train Loss: 0.5914, Val Loss: 0.6956, F1 Score: 0.4478
[cuda] Epoch 11/50, Train Loss: 0.6023, Val Loss: 0.7309, F1 Score: 0.4286
[cuda] Epoch 12/50, Train Loss: 0.5768, Val Loss: 0.6614, F1 Score: 0.5672
[cuda] Epoch 13/50, Train Loss: 0.5857, Val Loss: 0.6413, F1 Score

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[cuda] Epoch 2/50, Train Loss: 0.7068, Val Loss: 0.6963, F1 Score: 0.0000
[cuda] Epoch 3/50, Train Loss: 0.7135, Val Loss: 0.6961, F1 Score: 0.0513
[cuda] Epoch 4/50, Train Loss: 0.6989, Val Loss: 0.6997, F1 Score: 0.0930
[cuda] Epoch 5/50, Train Loss: 0.7002, Val Loss: 0.6970, F1 Score: 0.2000
[cuda] Epoch 6/50, Train Loss: 0.6984, Val Loss: 0.6906, F1 Score: 0.4407
[cuda] Epoch 7/50, Train Loss: 0.6945, Val Loss: 0.6889, F1 Score: 0.4000
[cuda] Epoch 8/50, Train Loss: 0.6869, Val Loss: 0.6966, F1 Score: 0.4545
[cuda] Epoch 9/50, Train Loss: 0.6856, Val Loss: 0.6967, F1 Score: 0.5294
[cuda] Epoch 10/50, Train Loss: 0.6833, Val Loss: 0.6771, F1 Score: 0.4194
[cuda] Epoch 11/50, Train Loss: 0.6865, Val Loss: 0.6891, F1 Score: 0.4928
[cuda] Epoch 12/50, Train Loss: 0.6791, Val Loss: 0.6985, F1 Score: 0.3729
[cuda] Epoch 13/50, Train Loss: 0.6703, Val Loss: 0.6987, F1 Score: 0.4444
[cuda] Epoch 14/50, Train Loss: 0.6672, Val Loss: 0.6913, F1 Score: 0.4333
[cuda] Epoch 15/50, Train Loss: 0

In [ ]:
best_params = {"learning_rate": 0.01, "weight_decay": 0.001}
best_params

{'learning_rate': 0.01, 'weight_decay': 0.001}

Run 5-fold cross validation

In [ ]:
# Run the 5-fold cross-validation
metrics = cross_validation(best_params, train_val_dataset, test_loader, num_epochs=50, folds=5)

# Calculate average and standard deviation of metrics across folds
avg_metrics = metrics.mean(axis=0)
std_metrics = metrics.std(axis=0)

print(f"\nAverage metrics over 5 folds in test set:\n"
      f"Accuracy: {avg_metrics[0]:.4f} ± {std_metrics[0]:.4f}\n"
      f"Precision: {avg_metrics[1]:.4f} ± {std_metrics[1]:.4f}\n"
      f"Recall: {avg_metrics[2]:.4f} ± {std_metrics[2]:.4f}\n"
      f"F1 Score: {avg_metrics[3]:.4f} ± {std_metrics[3]:.4f}\n"
      f"Training Time per Fold: {avg_metrics[4]:.2f} ± {std_metrics[4]:.2f} seconds")


Starting fold 1/5
[cuda] Epoch 1/50, Train Loss: 0.7036, Val Loss: 0.7135, F1 Score: 0.5189
[cuda] Epoch 2/50, Train Loss: 0.7185, Val Loss: 0.7073, F1 Score: 0.5882
[cuda] Epoch 3/50, Train Loss: 0.7035, Val Loss: 0.7140, F1 Score: 0.5756
[cuda] Epoch 4/50, Train Loss: 0.6874, Val Loss: 0.7057, F1 Score: 0.6204
[cuda] Epoch 5/50, Train Loss: 0.6863, Val Loss: 0.6921, F1 Score: 0.6138
[cuda] Epoch 6/50, Train Loss: 0.6731, Val Loss: 0.6947, F1 Score: 0.5765
[cuda] Epoch 7/50, Train Loss: 0.6491, Val Loss: 0.6917, F1 Score: 0.5926
[cuda] Epoch 8/50, Train Loss: 0.6270, Val Loss: 0.6735, F1 Score: 0.6098
[cuda] Epoch 9/50, Train Loss: 0.6303, Val Loss: 0.6680, F1 Score: 0.5926
[cuda] Epoch 10/50, Train Loss: 0.6106, Val Loss: 0.6448, F1 Score: 0.6053
[cuda] Epoch 11/50, Train Loss: 0.6002, Val Loss: 0.6368, F1 Score: 0.6709
[cuda] Epoch 12/50, Train Loss: 0.5983, Val Loss: 0.6333, F1 Score: 0.6122
[cuda] Epoch 13/50, Train Loss: 0.6187, Val Loss: 0.6594, F1 Score: 0.5541
[cuda] Epoch 14

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[cuda] Epoch 2/50, Train Loss: 0.7057, Val Loss: 0.7072, F1 Score: 0.0000
[cuda] Epoch 3/50, Train Loss: 0.7020, Val Loss: 0.7151, F1 Score: 0.0241
[cuda] Epoch 4/50, Train Loss: 0.6878, Val Loss: 0.7061, F1 Score: 0.0889
[cuda] Epoch 5/50, Train Loss: 0.6772, Val Loss: 0.6954, F1 Score: 0.3119
[cuda] Epoch 6/50, Train Loss: 0.6694, Val Loss: 0.6831, F1 Score: 0.4000
[cuda] Epoch 7/50, Train Loss: 0.6395, Val Loss: 0.6766, F1 Score: 0.4361
[cuda] Epoch 8/50, Train Loss: 0.6348, Val Loss: 0.6789, F1 Score: 0.5714
[cuda] Epoch 9/50, Train Loss: 0.6217, Val Loss: 0.6632, F1 Score: 0.5315
[cuda] Epoch 10/50, Train Loss: 0.6269, Val Loss: 0.6618, F1 Score: 0.5263
[cuda] Epoch 11/50, Train Loss: 0.6009, Val Loss: 0.6624, F1 Score: 0.4500
[cuda] Epoch 12/50, Train Loss: 0.6068, Val Loss: 0.6834, F1 Score: 0.4640
[cuda] Epoch 13/50, Train Loss: 0.5950, Val Loss: 0.6747, F1 Score: 0.4960
[cuda] Epoch 14/50, Train Loss: 0.6153, Val Loss: 0.6488, F1 Score: 0.5484
[cuda] Epoch 15/50, Train Loss: 0